# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
import numpy as np # Mengimpor library numpy untuk numerik
import pandas as pd  # Mengimpor pustaka pandas untuk manipulasi dan analisis data
import matplotlib.pyplot as plt # Mengimpor maplotlib untuk visualisasi grafik
import seaborn as sns # Mengimpor seaborn untuk visualisasi data
from yellowbrick.cluster import KElbowVisualizer  # Mengimpor KElbowVisualizer untuk visualisasi metode Elbow
from sklearn.cluster import KMeans, DBSCAN  # Mengimpor algoritma KMeans dan DBSCAN untuk clustering
from sklearn.metrics import silhouette_score  # Mengimpor silhouette_score untuk mengevaluasi hasil clustering
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV # Mengimpor train test split dan randomized search untuk memilih model terbaik
from sklearn.feature_selection import SelectKBest, mutual_info_classif, RFE # Mengimpor feature selection untuk meningkatkan akurasi
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler # Mengimpor encoder, standarisasi, dan normalisasi untuk preprocessing
from sklearn.neighbors import KNeighborsClassifier # Mengimpor algoritma KNN
from sklearn.tree import DecisionTreeClassifier # Mengimpor algoritma Decision Tree
from sklearn.ensemble import RandomForestClassifier # Mengimpor algoritma Random Forest
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.decomposition import PCA # Mengimpor PCA untuk visualisasi hasil clustering

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
# memuat dataset hasil clustering
df = pd.read_csv('dataset_clustering.csv')

# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [3]:
df.head()

,Customer_Type,Unit_Price,Quantity,Discount,Total_Price,Cluster
0,B2B,1.87,74.0,0.10,124.54,1
1,B2B,1.87,18.0,0.05,31.98,1
2,B2C,1.64,4.0,0.00,6.56,0
3,B2C,1.74,5.0,0.00,8.70,0
4,B2C,1.80,15.0,0.00,27.00,0


In [4]:
# buat instance LabelEncoder
label_encoder = LabelEncoder()

# encode kolom kategorikal
df[['Customer_Type']] = df[['Customer_Type']].apply(label_encoder.fit_transform)

# tampilkan dataframe untuk memastikan encoding telah diterapkan
df.head()

,Customer_Type,Unit_Price,Quantity,Discount,Total_Price,Cluster
0,0,1.87,74.0,0.10,124.54,1
1,0,1.87,18.0,0.05,31.98,1
2,1,1.64,4.0,0.00,6.56,0
3,1,1.74,5.0,0.00,8.70,0
4,1,1.80,15.0,0.00,27.00,0


In [5]:
# buat instance MinMaxScaler
scaler = MinMaxScaler()

# normalisasi semua kolom numerik
numeric_cols = ['Unit_Price', 'Quantity', 'Discount', 'Total_Price']
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

df.head()

,Customer_Type,Unit_Price,Quantity,Discount,Total_Price,Cluster
0,0,0.252459,0.737374,0.666667,0.626210,1
1,0,0.252459,0.171717,0.333333,0.159490,1
2,1,0.214754,0.030303,0.000000,0.031313,0
3,1,0.231148,0.040404,0.000000,0.042104,0
4,1,0.240984,0.141414,0.000000,0.134379,0


In [6]:
# memisahkan fitur (X) dan target (y)
X = df.drop(['Cluster'], axis=1)  
y = df['Cluster'] 

# membagi dataset menjadi training dan testing 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# memastikan split diterapkan
print(f"Training set shape: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Test set shape: X_test={X_test.shape}, y_test={y_test.shape}")

Training set shape: X_train=(3316, 5), y_train=(3316,)
Test set shape: X_test=(829, 5), y_test=(829,)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [7]:
# inisialisasi klasifikasi
knn = KNeighborsClassifier().fit(X_train, y_train)
dt = DecisionTreeClassifier().fit(X_train, y_train)

print("Model training selesai.")

Model training selesai.


Tulis narasi atau penjelasan algoritma yang Anda gunakan.

1. K-Nearest Neighbors: Algoritma ini digunakan untuk mengklasifikasikan data baru berdasarkan kedekatannya dengan data yang sudah diberi label dalam dataset pelatihan
2. Decision Tree: Algoritma dengan bentuk mirip pohon yang setiap cabang mewakili keputusan atau percabangan dari data berdasarkan fitur-fitur yang ada. 
3. Random Forest: Algoritma ensemble learning yang menggabungkan beberapa Decision Tree untuk meningkatkan akurasi prediksi dan mengurangi risiko overfitting.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [8]:
# fungsi untuk mengevaluasi dan mengembalikan hasil sebagai kamus
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    results = {
        'Confusion Matrix': cm,
        'True Positive (TP)': tp,
        'False Positive (FP)': fp,
        'False Negative (FN)': fn,
        'True Negative (TN)': tn,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred)
    }
    return results
 
# mengevaluasi setiap model dan mengumpulkan hasilnya
results = {
    'K-Nearest Neighbors (KNN)': evaluate_model(knn, X_test, y_test),
    'Decision Tree (DT)': evaluate_model(dt, X_test, y_test)
}
 
# buat DataFrame untuk meringkas hasil
summary_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])
 
# isi DataFrame dengan hasil
rows = []
for model_name, metrics in results.items():
    rows.append({
        'Model': model_name,
        'Accuracy': metrics['Accuracy'],
        'Precision': metrics['Precision'],
        'Recall': metrics['Recall'],
        'F1-Score': metrics['F1-Score']
    })
 
# konversi daftar kamus ke DataFrame
summary_df = pd.DataFrame(rows)
 
# tampilkan DataFrame
print(summary_df)

                       Model  Accuracy  Precision  Recall  F1-Score
0  K-Nearest Neighbors (KNN)       1.0        1.0     1.0       1.0
1         Decision Tree (DT)       1.0        1.0     1.0       1.0


Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.
1. Akurasi: Kedua algoritma menunjukkan akurasi 100%, dimana model terlalu cocok dengan set pelatihan, mungkin karena random sample yang diambil tidak memberikan gambaran yang luas terhadap data lainnya.
2. Precision, Recall, dan F1-Score: Kedua algoritma juga memilik score 1.0, karena model sangat cocok dengam set pelatihan, bukannya memahami pola, tetapi model cenderung menghapal data.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [9]:
# evaluasi awal model tanpa tuning
y_pred = knn.predict(X_test)
y_pred = dt.predict(X_test)
initial_mse = mean_squared_error(y_test, y_pred)
print(f"Initial MSE on test set (without tuning): {initial_mse:.2f}")

Initial MSE on test set (without tuning): 0.00


In [10]:
# hyperparameter tuning untuk KNN
knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'weights': ['uniform', 'distance']
    }

# GridSearchCV untuk KNN
knn_grid = GridSearchCV(estimator=knn, param_grid=knn_param_grid, cv=3, n_jobs=-1, scoring='accuracy')
knn_grid.fit(X_train, y_train)

best_knn = knn_grid.best_estimator_
y_pred = best_knn.predict(X_test)

# output hasil terbaik
print("Best K-Nearest Neighbors parameters:", knn_grid.best_params_)

Best K-Nearest Neighbors parameters: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}


In [11]:
# hyperparameter tuning untuk Decision Tree
dt_param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [None, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 3, 4],
    'splitter': ['best', 'random']
    }

# GridSearchCV untuk Decision Tree
dt_grid = GridSearchCV(estimator=dt, param_grid=dt_param_grid, cv=3, n_jobs=-1, scoring='accuracy')
dt_grid.fit(X_train, y_train)

best_dt = dt_grid.best_estimator_
y_pred = best_dt.predict(X_test)

# output hasil terbaik
print("Best Decision Tree parameters:", dt_grid.best_params_)

Best Decision Tree parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [12]:
# fungsi untuk mengevaluasi dan mengembalikan hasil sebagai kamus
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    results = {
        'Confusion Matrix': cm,
        'True Positive (TP)': tp,
        'False Positive (FP)': fp,
        'False Negative (FN)': fn,
        'True Negative (TN)': tn,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred)
    }
    return results
 
# mengevaluasi setiap model dan mengumpulkan hasilnya
results = {
    'K-Nearest Neighbors (KNN)': evaluate_model(best_knn, X_test, y_test),
    'Decision Tree (DT)': evaluate_model(best_dt, X_test, y_test)
}
 
# buat DataFrame untuk meringkas hasil
summary_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])
 
# isi DataFrame dengan hasil
rows = []
for model_name, metrics in results.items():
    rows.append({
        'Model': model_name,
        'Accuracy': metrics['Accuracy'],
        'Precision': metrics['Precision'],
        'Recall': metrics['Recall'],
        'F1-Score': metrics['F1-Score']
    })
 
# konversi daftar kamus ke DataFrame
summary_df = pd.DataFrame(rows)
 
# tampilkan DataFrame
print(summary_df)

                       Model  Accuracy  Precision  Recall  F1-Score
0  K-Nearest Neighbors (KNN)       1.0        1.0     1.0       1.0
1         Decision Tree (DT)       1.0        1.0     1.0       1.0


Evaluasi Model Setelah Tuning
1. Karena sebelum tuning akurasi sudah 100%, maka setelah melakukan tunig hasilnya tidak ada perubahan.

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Evaluasi setelah tuning tidak ada perubahan, karena sebelum tuning akurasi pada kedua model telah sempurna (1.0).
2. Model yang yang saya lakukan mengalami overfitting, mungkin ini disebabkan adanya kesalahan pada preprocessing clustering karena dataset terlalu besar, jadi saya mengambil subset secara acak sebanyak 10.000 yang mana mungkin sampling tersebut tidak mewakili keseluruhan dataset dan pada saat di cek terdapat duplikasi karena tidak melibatkan kolom ID.
3. Rekomendasi yang bisa dilakukan:
   - Melakukan preprocessing dengan benar pada saat clustering.
   - Dapat melakukan penanganan overfitting dan underfitting.
   - Jika melakukan sampling, past
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.